In [1]:
import torch
import transformers
from transformers import AutoTokenizer
from argparse import Namespace

In [6]:
# change path_to_model_on_disk and model_ckpt

# Options for path_to_model_on_disk:
# "/data3/mmendieta/models/xlmt_finetuned_twitter/worldly-blaze-2/epoch_14"
# "/data3/mmendieta/models/smallLabse_finetuned_twitter/electric-glitter-32/epoch_15"
# "/data3/mmendieta/models/labse_finetuned_twitter/dazzling-violet-5/epoch_19"

# Options for model_ckpt
# "cardiffnlp/twitter-xlm-roberta-base"
# "setu4993/LaBSE"
# "setu4993/smaller-LaBSE"

config = {
    "cuda_device": 12,
    "path_to_model_on_disk": "/data3/mmendieta/models/xlmt_finetuned_twitter/", 
    "model_ckpt": "cardiffnlp/twitter-xlm-roberta-base",
    "max_length": 32
}

args = Namespace(**config)

In [7]:
# Instantiate the tokenizer
model_ckpt = args.model_ckpt
tokenizer = AutoTokenizer.from_pretrained(model_ckpt,
                                         model_max_length=args.max_length
                                         )

In [9]:
tokenizer.save_pretrained(args.path_to_model_on_disk)

('/data3/mmendieta/models/xlmt_finetuned_twitter/tokenizer_config.json',
 '/data3/mmendieta/models/xlmt_finetuned_twitter/special_tokens_map.json',
 '/data3/mmendieta/models/xlmt_finetuned_twitter/sentencepiece.bpe.model',
 '/data3/mmendieta/models/xlmt_finetuned_twitter/added_tokens.json',
 '/data3/mmendieta/models/xlmt_finetuned_twitter/tokenizer.json')

### Hugging Face hub

In [ ]:
from huggingface_hub import notebook_loginh, Repository, get_full_repo_name
notebook_login()

In [ ]:
from huggingface_hub import Repository, get_full_repo_name, list_repo_refs

In [ ]:
model_repo_hub = "m2im/XLM-T_finetuned_violence_twitter" 

In [ ]:
hf_repo = Repository("/data3/mmendieta/test/smallLabse_finetuned_twitter", clone_from="m2im/smallLabse_finetuned_twitter/electric-glitter-32")

In [ ]:
list_repo_refs("m2im/smallLabse_finetuned_twitter")